In [4]:
%matplotlib inline
import pyspeckit
import numpy as np
from astropy import units as u
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.convolution import convolve, Box1DKernel
from c_normalize import c_normalize
import ipywidgets as widgets
import glob
from IPython.display import display


plt.style.use("seaborn-darkgrid")

# to get widgets to work in jupyter lab: jupyter labextension install @jupyter-widgets/jupyterlab-manager
# need to install nodejs as well

## Visualizing Fit Performance on the Master Line List
With the Fe lines, fit performance was assessed in a manual fashion. With more than double the lines for the master list, it's worthwhile to come up with a more elegant solution for assessing fit performance. It will still need to be manual, but rather than looping through all of the spectra, I'd like to try building a jupyter widget that plugs into our dataset and allows for more interactive inspection of the fit

In [40]:
# Widgets
files = glob.glob("../pydata/ew_known/inputs/*wavsoln.fits")
files = [file.split("/")[-1]for file in files]

f_drop = widgets.Dropdown(
    options=files,
    value='col110_1134red_oned_25jan14_wavsoln.fits',
    description='File:',
    disabled=False,
)

mf_drop = widgets.Dropdown(
    options=['1','2','3'],
    value='1',
    description='Num. fits:',
    disabled=False,
)
fit_text = []
for i in range(int(mf_drop.value)):
    fit_group = []
    fit_group.append(widgets.Text(
    value='-0.3',
    placeholder='Type something',
    description=f'Amp {i+1}:',
    disabled=False
))
    
    fit_group.append(widgets.Text(
    value='0',
    placeholder='Type something',
    description=f'Offset {i+1}:',
    disabled=False
))

    
    fit_group.append(widgets.Text(
    value='0.15',
    placeholder='Type something',
    description=f'Width {i+1}:',
    disabled=False
))
    fit_text.append(fit_group)


display(f_drop, mf_drop)
for group in fit_text:
    display(*group)

Dropdown(description='File:', options=('col110_1134red_oned_25jan14_wavsoln.fits', 'col110_2119red_oned_26jan1…

Dropdown(description='Num. fits:', options=('1', '2', '3'), value='1')

Text(value='-0.3', description='Amp 1:', placeholder='Type something')

Text(value='0', description='Offset 1:', placeholder='Type something')

Text(value='0.15', description='Width 1:', placeholder='Type something')